In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import os
print(os.listdir("../input"))

In [ ]:
from fastai.text import *
from fastai.imports import *
from fastai.text import *
from fastai import *

In [ ]:
test = pd.read_csv('/kaggle/input/janata-data/test.csv')
train = pd.read_csv('/kaggle/input/janata-data/train.csv')
game = pd.read_csv('/kaggle/input/janata-data/game_overview.csv')

In [ ]:
train=train.merge(game, on='title', how='left')
test=test.merge(game, on='title', how='left')

In [ ]:
train.head(2)

In [ ]:
test.head(2)

In [ ]:
train['text']=train['user_review']+' '+ train['title']+' '+train['developer']+' '+train['publisher']+' '+train['tags']+' '+train['overview']

In [ ]:
test['text']=test['user_review']+' '+ test['title']+' '+test['developer']+' '+test['publisher']+' '+test['tags']+' '+test['overview']

In [ ]:
del train['user_review']
del train['title']
del train['developer']
del train['publisher']
del train['tags']
del train['overview']
del train['year']
del train['review_id']

In [ ]:
cc=test['review_id']

In [ ]:
del test['user_review']
del test['title']
del test['developer']
del test['publisher']
del test['tags']
del test['overview']
del test['year']
del test['review_id']

In [ ]:
df=train['user_suggestion']
del train['user_suggestion']
train['user_suggestion']=df

In [ ]:
train['text'][0]

In [ ]:
import seaborn as sns

sns.countplot(x='user_suggestion', data=train)

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'@[a-zA-Z0-9_]+', '', text)   
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)   
    text = re.sub(r'www.[^ ]+', '', text)  
    text = re.sub(r'[a-zA-Z0-9]*www[a-zA-Z0-9]*com[a-zA-Z0-9]*', '', text)  
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = [token for token in text.split() if len(token) > 2]
    text = ' '.join(text)
    return text

train['text'] = train['text'].apply(clean_text)
test['text'] = test['text'].apply(clean_text)

In [ ]:
def random_seed(seed_value):
    import random 
    random.seed(seed_value)  
    import numpy as np
    np.random.seed(seed_value)  
    import torch
    torch.manual_seed(seed_value)  
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True   
        torch.backends.cudnn.benchmark = False

In [ ]:
path = Path('../input/janata-data/')
path.ls()

In [ ]:

from sklearn.metrics import accuracy_score 
y_pred_totcb = []
from sklearn.model_selection import KFold, RepeatedKFold
fold = KFold(n_splits=15, shuffle=True, random_state=0)
i=1

for train_index, test_index in fold.split(train):
    
    train_df = train.iloc[train_index]
    valid_df = train.iloc[test_index]

    random_seed(10)
    
    data_lm = TextLMDataBunch.from_df(Path(path), train_df, valid_df, test, text_cols=[0], bs=32)
    data_clas = TextClasDataBunch.from_df(Path(path), train_df, valid_df, test, text_cols=[0], label_cols=1, bs=32)
    
    learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.4, model_dir='/tmp/model/')
    learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))
    learn.unfreeze()
    learn.fit_one_cycle(9, 1e-3, moms=(0.8,0.7))
    learn.save_encoder('model_enc')
    
    learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.4, model_dir='/tmp/model/')
    learn.load_encoder('model_enc')
    learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))
    learn.freeze_to(-2)
    learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))
    learn.freeze_to(-3)
    learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))
    learn.unfreeze()
    learn.fit_one_cycle(5, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))
   
    log_preds, test_labels = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
    preds = np.argmax(log_preds, 1)
    y_pred_totcb.append(preds)
    print(f'fold {i} completed')
    i = i+1

In [ ]:
y_pred_totcb

In [ ]:
submission = pd.DataFrame({
        "review_id": cc,
        "user_suggestion": y_pred_totcb[0]
    })
submission.to_csv('./Av_nlp.csv', index=False)
print(submission)

# 2nd Method

In [ ]:

import re
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
warnings.filterwarnings("ignore")

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk_stopwords = stopwords.words('english')

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'@[a-zA-Z0-9_]+', '', text)   
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)   
    text = re.sub(r'www.[^ ]+', '', text)  
    text = re.sub(r'[a-zA-Z0-9]*www[a-zA-Z0-9]*com[a-zA-Z0-9]*', '', text)  
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = [token for token in text.split() if len(token) > 2]
    text = ' '.join(text)
    return text

train['text'] = train['text'].apply(clean_text)
test['text'] = test['text'].apply(clean_text)

In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(train['text'], train['text'], test_size=0.1, stratify=train['user_suggestion'], 
                                                random_state=1)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word', stop_words=None, token_pattern='(?u)\\b\\w\\w+\\b', ngram_range=(1, 1), max_df=1.0, 
                             min_df=3, max_features=None, binary=False, norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

In [ ]:
X_train_tfidf = tfidf_vect.fit_transform(X_train)
X_cv_tfidf = tfidf_vect.transform(X_cv)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier

In [ ]:
sgd = SGDClassifier(loss='log', max_iter=100, random_state=101, class_weight='balanced')
ovr = OneVsRestClassifier(sgd)
ovr.fit(X_train_tfidf, y_train)
y_pred_class = ovr.predict(X_cv_tfidf)

In [ ]:
print('f1_score       :', f1_score(y_cv, y_pred_class, average='macro'))
print('accuracy score :', accuracy_score(y_cv, y_pred_class))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word', stop_words=None, token_pattern='(?u)\\b\\w\\w+\\b', ngram_range=(1, 1), max_df=1.0, 
                             min_df=3, max_features=None, binary=False, norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

In [ ]:
full_text = list(train['text'].values) + list(test['text'].values)
tfidf_vect.fit(full_text)

X_train_tfidf = tfidf_vect.transform(train['text'])
X_test_tfidf = tfidf_vect.transform(test['text'])

y_train = train['user_suggestion']

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss='log', max_iter=200, random_state=0, class_weight='balanced')
ovr = OneVsRestClassifier(sgd)
ovr.fit(X_train_tfidf, y_train)
y_pred_class = ovr.predict(X_test_tfidf)
y_pred_class

In [ ]:
submission = pd.DataFrame({
        "review_id": cc,
        "user_suggestion": y_pred_class
    })
submission.to_csv('./Av_nlp.csv', index=False)
print(submission)